In [1]:
#https://gist.github.com/armfazh/284eb8b461c083a8c99e728164bfaf45

#order = 3450873173395281893717377931138512760570940988862252126328087024741343
#Gx = 0x17232ba853a7e731af129f22ff4149563a419c26bf50a4c9d6eefad6126
#Gy = 0x1db537dece819b7f70f555a67c427a8cd9bf18aeb9b56e0c11056fae6a3

F2.<t> = GF(2)[]
m = 233
p_t = t ** m + t ** 74 +1
F2m = GF(2^m,'t',modulus=p_t)

Hex = lambda x : map(hex,c)
def int2F2m(n):
    return F2m.fetch_int(n)
def F2m2int(a):
    return ZZ("".join(map(str,a.polynomial().coefficients(False)))[::-1],base=2)
def print_point(P):
    return map(lambda x: hex(F2m2int(x)),P)

# Weierstraass elliptic curve defined by
# E : y^2 + a_1xy + a_3y = x^3 + a_2x^2 +a_4x + a_6

# A binary curve over F_{2^m} is defined by:
# E_{a,b} : y^2 + xy + a_3y = x^3 + a_2x^2 +a_4x + a_6
# where a_1=1, a_3=0,a_2=a,a_4=0, and a_6=b.

# From NIST REC ECC
# A random binary curve
# a = 1
# A Koblitz binary curve
# a = {0,1} and b = 1

a = F2m.one()
b = int2F2m(0x066647ede6c332c7f8c0923bb58213b333b20e9ce4281fe115f7d8f90ad)
E = EllipticCurve(F2m,[F2m.one(),a,F2m.zero(),F2m.zero(),b])
Gx = int2F2m(0x0fac9dfcbac8313bb2139f1bb755fef65bc391f8b36f8f8eb7371fd558b)
Gy = int2F2m(0x1006a08a41903350678e58528bebf8a0beff867a7ca36716f7e01f81052)
G = E([Gx,Gy,1])
order = 6901746346790563787434755862277025555839812737345013555379383634485463
assert(is_prime(order))



def Weierstrass_to_twistedmu4(P):
    x,y,z = P
    return [x**2,x**2+y,z,x**2+y+x]

def twistedmu4_to_Weierstrass(P):
    X0,X1,X2,X3 = P
    return [ X1+X3, X0+X1,X2 ]

def in_twistedmu4(P):
    X0,X1,X2,X3 = P
    return X0**2+b*X2**2 == X1*X3+a*X0*X2 and  X1**2+X3**2 == X0*X2


for i in range(10000):
    k = randint(0,order)
    P = k*G
    S = Weierstrass_to_twistedmu4(P)
    U = twistedmu4_to_Weierstrass(S)
    assert in_twistedmu4(S) and U == P

AssertionError: 

In [9]:
#https://stackoverflow.com/questions/56371574/exponentiation-on-a-point-on-elliptic-curve-unreasonably-fast-in-sagemath
import time
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951 
order = 115792089210356248762697446949407573529996955224135760342422259061068512044369 
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b 
F = GF(p) 
E = EllipticCurve(F, [-3,b]) 
runs = 10000 
G = E.abelian_group()
# G = E.random_element()

F2 = GF(order) 
exponent = [F2.random_element() for i in range(runs)]
e2 = [G.random_element() for i in range(runs)]

t1 = time.time() 
for i in range(runs):   
      e = Integer(exponent[i])*e2[i] 
t2 = time.time() 
print("Time per operation = ", (t2 - t1)/runs , " seconds")

e1 = [G.random_element() for i in range(runs)] 
e2 = [G.random_element() for i in range(runs)] 
t1 = time.time() 
for i in range(runs):   
         e = e1[i]+e2[i] 
t2 = time.time() 
print("Time per operation = ", (t2 - t1)/runs , " seconds")

Time per operation =  3.747725486755371e-06  seconds
Time per operation =  3.436565399169922e-06  seconds


In [10]:
#https://gist.github.com/armfazh/71527a94972e7f80f5aed074c9b6afe7

# @author Armando Faz
# @brief Instance of GLS254 Curve by CHES2013
#        Oliveira, López, Aranha, Rodríguez

F.<t> = GF(2)[]
Fq = GF(2^127,'t',modulus=t ** 127 + t ** 63 + 1)
R.<u> = Fq[]
Fq2 = Fq.extension(u ** 2 + u + 1)

Hex = lambda X : map(hex,X)

def intToFq(n):
    return Fq.fetch_int(n)

def FqToint(s):
    return sum([ ZZ(bit)*2**i for i,bit in enumerate(s.polynomial().coefficients(False))  ])

def Fq2Toint(s):
    return [ FqToint(s[0]), FqToint(s[1]) ]

def print_point(P):
	print("P.x: " , Hex(Fq2Toint(P[0])))
	print("P.y: " , Hex(Fq2Toint(P[1])))
    

# Weierstraass binary elliptic curve defined by
# E_{a,b} : y^2 + a_1xy + a_3y = x^3 + a_2x^2 +a_4x + a_6
# where a_1=1, a_3=0,a_2=a,a_4=0, and a_6=b.

a = Fq2(u)
b = Fq2(intToFq(0x59C8202CB9E6E0AE2E6D944FA54DE7E5))
E = EllipticCurve(Fq2,[Fq2.one(),a,Fq2.zero(),Fq2.zero(),b])
Gx = Fq2 ( \
	   intToFq(0x203B6A93395E0432344038B63FBA32DE) \
	+u*intToFq(0x78E51FD0C310696D5396E0681AA10E0D) )
Gl = Fq2 ( \
	   intToFq(0x5BD7653482085F55DEB59C6137074B50) \
	+u*intToFq(0x7F90D98B1589A17F24568FA5A1033946) )
Gy = (Gl-Gx)*Gx

G = E([Gx,Gy])
order = 0x1FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFDAC40D1195270779877DABA2A44750A5
cofactor=2
assert(is_prime(order))

In [11]:
#https://ask.sagemath.org/question/23707/binary-field-multiplication/
# Use two elements of GF(2^4) represented by a hexadecimal coefficient bit vector.
C = Integer(0xa); # a = x^3 + x = 1010(bin) = 10(dec) = a(hex)
H = Integer(0xa);
X1_exp = Integer(0x8); # Expected product = x^3 = 1000(bin) = 8(dec) = 8(hex)
# Create the binary finite field using the given irreducible polynomial p(x) = x^4 + x + 1;
P2.<x> = GF(2)[];
p = x^4 + x + 1;
GF16 = GF(2^4, 'x', modulus=p); GF16
# Create the field elements from the hexadecimal representation and multiply them.
C_bf = GF16._cache.fetch_int(C);
H_bf = GF16._cache.fetch_int(H);
X1_bf = C_bf * H_bf;
# Convert the field element back into a HEX representation containing the coefficients only.
X1_bitvec = ''.join(map(str,X1_bf.polynomial().coeffs()));
X1 = Integer(str(X1_bitvec)[::-1], base=2);
# Check whether the result is correct.
X1 == X1_exp

AttributeError: 'sage.rings.polynomial.polynomial_gf2x.Polynomial_GF2X' object has no attribute 'coeffs'